In [10]:
import numpy as np
import pandas as pd
import os
import warnings
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree
from ClassificadorAlfa import ClassificadorAlfa
warnings.filterwarnings('ignore')

In [11]:
df = pd.read_csv(os.path.join('data', 'healthcare-dataset-stroke-data.csv'))

## Hipótese 1

Para a hipótese 1, vamos testar todos os dados contidos no DataFrame, sem alterações, porém apenas os dados que podem ser tratados como Booleanos. Vamos utilizar as variáveis categóricas para realizar o teste. A hipótese nula, no caso, seria montar um classificador que irá sempre prever os dados como sendo o mais frequente $(x)$, ou seja, será a acurácia medida ao dividir tal valor pelo total dos dados contidos no DataFrame $(N)$:

$$
Acurácia = \frac{x}{N}
$$

Portanto, vamos calcular, ao final, a acurácia da hipótese nula e verificar como a nossa hipótese, utilizando todas as variáveis categóricas, se comporta com relação a ela.

In [12]:
# Separando target e features
try:
    df_ = df.drop(['id', 'bmi', 'age', 'avg_glucose_level'], axis=1)
except:
    pass

X = df_.drop('stroke', axis=1)
y = df_['stroke']

In [13]:
# Obtendo variáveis categoricas
object_features = [feature for feature in X.columns if X[feature].dtype == 'O']
int_features = [feature for feature in X.columns if X[feature].dtype == 'int64']
categorical_features = object_features + int_features

In [14]:
# Obtendo variáveis dummy
X = X[categorical_features]

X = pd.get_dummies(X)
X.hypertension = X.hypertension.astype('bool')
X.heart_disease = X.heart_disease.astype('bool')

X_names = X.columns

y = y.replace(0,-1)

X = X.to_numpy()
y = y.to_numpy()

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5)
X_train = X_train.astype('float64')
y_train = y_train.astype('float64')
X_test = X_test.astype('float64')
y_test = y_test.astype('float64')

In [16]:
# Inicializando os parâmetros do modelo
a = np.random.randn(X_train.shape[1], 1)
b = 1.0

X_train = X_train.T
y_train = y_train.T

parametros = [a, b, X_train, y_train]
learning_rate = 0.0001
num_iteracoes = 50000

In [17]:
classificador = ClassificadorAlfa(learning_rate, num_iteracoes, parametros)

In [18]:
# Treinando o modelo
a, b = classificador.treinar()
a, b

KeyboardInterrupt: 

In [ ]:
# Fazendo previsões
ypred = a.T @ X_test.T + b
ypred

In [ ]:
acuracia = ClassificadorAlfa.acuracia(y_test, ypred)
print(f'A acurácia do modelo foi de {acuracia*100:.2f}%')

In [ ]:
# Selecionando as features mais importantes
features = X_names
importances = pd.DataFrame(data=a, index=features, columns=['importance']).sort_values(by='importance', ascending=False)
importances

In [ ]:
# Obtendo a acurácia da hipótese nula
valor_mais_frequente = np.bincount(y).argmax()
acuracia_nula = np.mean(y == valor_mais_frequente)

print(f'A acurácia da hipótese nula foi de {acuracia_nula*100:.2f}%')
print(f'A acurácia do modelo foi de {acuracia*100:.2f}%')

### Utilizando um classificador de árvore de decisão

Em seguida, vamos comparar o nosso modelo com um classificador de árvore de decisão, comparando, novamente, as acurácias obtidas.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
classificador = DecisionTreeClassifier(criterion='entropy')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5)
X_train = X_train.astype('float64')
y_train = y_train.astype('float64')
X_test = X_test.astype('float64')
y_test = y_test.astype('float64')

In [ ]:
classificador.fit(X_train, y_train)

In [ ]:
classificador.predict(X_test)

In [ ]:
acuracia = classificador.score(X_test, y_test)

In [ ]:
print(f'A acurácia da hipótese nula foi de {acuracia_nula*100:.2f}%')
print(f'A acurácia do modelo foi de {acuracia*100:.2f}%')

In [ ]:
# Plotando a árvore de decisão
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 10))
plot_tree(classificador, filled=True, rounded=True, class_names=['Não teve AVC', 'Teve AVC'], feature_names=X_names)
plt.show()